In [8]:
from skimage.feature import hog
from skimage.feature import local_binary_pattern
import cv2 
import os
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

In [9]:
# method to create data set
def create_dataset(live_path, spoof_path, descriptor):
    
    # list to store extracted features of an image
    features = []
    
    # list to store class label, 1 for live, 0 for spoof
    labels = []    
    
    radius = 3
    
    # number of neighbors to consider for LBP
    n_points = 8 * radius 
    
    # sampling type for LBP
    METHOD = 'uniform'       
    
    path_array = [live_path, spoof_path]
    
    for path in path_array:
        
        # storing all images in a folder in a list 'files'
        files = os.listdir(path) 
        
        # loop through the images in the folder
        for img in files:
            
            # reading the image in grayscale using cv2
            img = cv2.imread(path + img, cv2.IMREAD_GRAYSCALE)
            
            # resizing the image so all images are of same size
            resized_img = cv2.resize(img, (300, 300))  
            
            # Extracting features of an image using LBP
            if descriptor == 'LBP':    
                lbp = local_binary_pattern(resized_img, n_points, radius, METHOD)
                
                 # Converting into 1-D array
                fd=lbp.flatten()
                
            # Extracting features of an image using HOG     
            else: 
                fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                                cells_per_block=(2, 2), visualize=True, multichannel=False)
                
            # label 1 for live images, 0 for spoof images
            class_identifier = 1         
            if 'spoof' in path:
                class_identifier = 0
                
             # appending exracted features to the list
            features.append(fd) 
            
            #adding corresponding class label to the list
            labels.append(class_identifier)  
           
                   
    return features,labels


In [5]:
# variables for differet folders
training_live_path = "/Users/deepshikhagupta/Downloads/Image Analysis and Computer Vision/Homework2/Training Biometrika Live/live/"
training_spoof_path = "/Users/deepshikhagupta/Downloads/Image Analysis and Computer Vision/Homework2/Training Biometrika Spoof/Training Biometrika Spoof/spoof/"
testing_live_path = "/Users/deepshikhagupta/Downloads/Image Analysis and Computer Vision/Homework2/Testing Biometrika Live/live/"
testing_spoof_path = "/Users/deepshikhagupta/Downloads/Image Analysis and Computer Vision/Homework2/Testing Biometrika Spoof/Testing Biometrika Spoof/spoof/"

In [6]:
# LBP 

# Training and testing datasets
lbp_x_trn,lbp_y_trn = create_dataset(training_live_path,training_spoof_path, 'LBP') 
lbp_x_tst,lbp_y_tst = create_dataset(testing_live_path,testing_spoof_path, 'LBP')

# Create and fit the model
lbp_clf = svm.SVC()
lbp_clf.fit(lbp_x_trn,lbp_y_trn)

# Predict on the test features, print the results
lbp_y_pred = lbp_clf.predict(lbp_x_tst)
print("LBP Accuracy: "+str(accuracy_score(lbp_y_tst, lbp_y_pred)))
print('\n')
print(classification_report(lbp_y_tst, lbp_y_pred))

LBP Accuracy: 0.935


              precision    recall  f1-score   support

           0       0.89      0.99      0.94       200
           1       0.99      0.88      0.93       200

    accuracy                           0.94       400
   macro avg       0.94      0.94      0.93       400
weighted avg       0.94      0.94      0.93       400



In [7]:
#HOG

# Training and testing datasets
hog_x_trn,hog_y_trn = create_dataset(training_live_path,training_spoof_path, 'HOG')
hog_x_tst,hog_y_tst = create_dataset(testing_live_path,testing_spoof_path, 'HOG')

# Create and fit the model
hog_clf = svm.SVC()
hog_clf.fit(hog_x_trn,hog_y_trn)

# Predict on the test features, print the results
hog_y_pred = hog_clf.predict(hog_x_tst)
print(" HOG Accuracy: "+str(accuracy_score(hog_y_tst, hog_y_pred)))
print('\n')
print(classification_report(hog_y_tst, hog_y_pred))

 HOG Accuracy: 0.8375


              precision    recall  f1-score   support

           0       0.77      0.95      0.85       200
           1       0.94      0.72      0.82       200

    accuracy                           0.84       400
   macro avg       0.86      0.84      0.84       400
weighted avg       0.86      0.84      0.84       400

